<a href="https://colab.research.google.com/github/Jodia03/project-/blob/main/PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install


In [ ]:
!pip install ipywidgets
#to enables `ipywidgets`
!jupyter nbextension enable --py widgetsnbextension

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.7 MB/s eta 0:00:00
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


# Modules

In [ ]:
#modules used
import pandas as pd
from ipywidgets import widgets
from IPython.display import display, HTML
import geopandas as gpd
import folium
import requests
from folium import IFrame
from ipywidgets import Button

# Main Code

In [ ]:
#loads the CSV data for courses
courses_url = 'https://raw.githubusercontent.com/Jodia03/project-/main/Degree_Programs.csv'
df = pd.read_csv(courses_url)

#loads the CSV data for universities
universities_url = 'https://raw.githubusercontent.com/Jodia03/project-/main/UNIVERSITIES%20-%20Sheet1.csv'
universities_df = pd.read_csv(universities_url)

#gets the degree programs for dropdown
name_list = df['DEGREE PROGRAM'].tolist()

In [ ]:
#gets the list of universities offering the selected course
def get_univ(course_name):
  mask = universities_df['DEGREE PROGRAMS'].fillna('').apply(lambda x: course_name in x)
  universities = universities_df[mask]['UNIVERSITIES'].tolist()
  locations = universities_df[mask]['LOCATION'].tolist()
  link = universities_df[mask]['WEBSITE LINK'].tolist()
  school_types = universities_df[mask]['SCHOOL TYPE'].tolist()
  return universities, locations, link, school_types

In [ ]:
#formats the legend
def format_legend(course):
  parts = course.split()
  if parts[0] in ['BS', 'BA', 'AB']: #to maintain the capitalization
    return f"{parts[0]} {' '.join(parts[1:]).title()}"
  else:
    return course.title()

In [ ]:
#generates HTML popup content
def generate_popup_html(city, count, universities, links):
  #integrate the link to university name
  universities_list = ''.join(f'<li><a href="{link}" target="_blank">{uni}</a></li>' for uni, link in zip(universities, links))
  html = f"""
  <style>
    .popup-header {{
      font-size: 22px; /*font size of the header*/
      color: black; /*font color of the header*/
    }}
    .popup-header u {{
        text-decoration: none; /*remove the underline*/
    }}
  </style>

  <!DOCTYPE html>

  <html>
    <h1 class="popup-header" align="center"><strong><u>{city}</u></strong></h1>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
    <link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" rel="stylesheet"/>

    <div class="panel-group" id="accordion" role="tablist" aria-multiselectable="true">
      <div class="panel panel-default">
          <div class="panel-heading" role="tab" id="headingOne">
            <h4 class="panel-title">
              <a role="button" data-toggle="collapse" data-parent="#accordion" href="#collapseOne" aria-expanded="false" aria-controls="collapseOne">
                <strong> <i class="fa fa-university"></i> List of Universities <span class="glyphicon glyphicon-menu-down" aria-hidden="true" style="float: right;"></span></strong>
              </a>
            </h4>
          </div>
          <div id="collapseOne" class="panel-collapse collapse in" role="tabpanel" aria-labelledby="headingOne">
            <div class="panel-body" align="left" style="font-size: 12px; padding-bottom: 4 !important; padding-top: 2 !important;">
                <ul>{universities_list}</ul>
            </div>
          </div>
        </div>
      <div class="panel panel-default">
        <div class="panel-heading" role="tab" id="headingTwo">
          <h4 class="panel-title">
            <a class="collapsed" role="button" data-toggle="collapse" data-parent="#accordion" href="#collapseTwo" aria-expanded="false" aria-controls="collapseTwo">
              <strong><i class="fa fa-book"></i> Number of Schools <span class="glyphicon glyphicon-menu-down" aria-hidden="true" style="float: right;"></span> </strong>
            </a>
          </h4>
          </div>
          <div id="collapseTwo" class="panel-collapse collapse" role="tabpanel" aria-labelledby="headingTwo">
            <div class="panel-body" align="left" style="font-size: 12px; padding-bottom: 2 !important; padding-top: 2 !important;">
              <div class="panel-body" style="padding-bottom: 2 !important; padding-top: 2 !important;">{count}</div>
            </div>
        </div>
      </div>
    </div>
    <script src="https://code.jquery.com/jquery-3.5.1.min.js"></script>
    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js"></script>
  </html>
  """
  return html

In [ ]:
#creates and displays the choropleth map
def create_choropleth_map(course, universities, cities, links, school_types):
  #access the gpkg file from Google Drive file
  file_url = 'https://drive.google.com/uc?export=download&id=1o3h0uZmn8cyTfRKCC_Usa-zA9bkQpmQW'
  output_file_path = 'metro_manila.gpkg'

  #downloads the file
  response = requests.get(file_url)
  with open(output_file_path, 'wb') as file:
    file.write(response.content)

  #reads the gpkg file
  metro_manila = gpd.read_file(output_file_path)
  metro_manila = metro_manila.sort_values('CODE')  #arranges the code alphabetically

  #adds a column for the name of cities, if it does not exist
  metro_manila.insert(1, 'City', ['City of Caloocan', 'City of Las Piñas',
                                      'City of Marikina', 'City of Malabon',
                                      'City of Mandaluyong', 'City of Makati',
                                      'City of Manila', 'City of Muntinlupa',
                                      'City of Navotas', 'City of Parañaque',
                                      'City of Pasig', 'City of Pasay',
                                      'City of Pateros', 'Quezon City',
                                      'City of San Juan', 'City of Taguig',
                                      'City of Valenzuela'], True)

  #counts the number of universities offering the course in each city
  location_counts = pd.Series(cities).value_counts().reset_index()
  location_counts.columns = ['City', 'Count']

  #counts the number of public universities in each city
  public_counts = pd.Series([city for city, school_type in zip(cities, school_types) if 'Public' in school_type]).value_counts().reset_index()
  public_counts.columns = ['City', 'Public_Count']

  #counts the number of private universities in each city
  private_counts = pd.Series([city for city, school_type in zip(cities, school_types) if 'Private' in school_type]).value_counts().reset_index()
  private_counts.columns = ['City', 'Private_Count']

  #merges with the geo-data
  metro_manila = metro_manila.merge(location_counts, on='City', how='left').fillna(0)
  metro_manila = metro_manila.merge(public_counts, on='City', how='left').fillna(0)
  metro_manila = metro_manila.merge(private_counts, on='City', how='left').fillna(0)

  #adds the universities to metro_manila DataFrame
  universities_df_temp = pd.DataFrame({'City': cities, 'University': universities, 'Link': links})
  universities_grouped = universities_df_temp.groupby('City').agg(list).reset_index()
  metro_manila = metro_manila.merge(universities_grouped, on='City', how='left')

  #ensures 'University' and 'Link' columns have lists even for cities with no data
  metro_manila['University'] = metro_manila['University'].apply(lambda x: x if isinstance(x, list) else [])
  metro_manila['Link'] = metro_manila['Link'].apply(lambda x: x if isinstance(x, list) else [])

  #creates a map centered on Metro Manila
  m = folium.Map(location=[14.5995, 120.9842], zoom_start=11, min_zoom=10.5, max_zoom=11)

  #creates a choropleth map for the number of universities offering the course
  university = folium.Choropleth(
      geo_data=metro_manila,
      name='All Universities',
      data=metro_manila,
      columns=['City', 'Count'],
      key_on='feature.properties.City',
      fill_color='Blues',
      fill_opacity=1,
      line_opacity=0.2,
      legend_name=f'Number of Universities offering {format_legend(course)}'
  ).add_to(m)

  #creates a choropleth map for public universities
  public_univ = folium.Choropleth(
      geo_data=metro_manila,
      name='Public Universities',
      data=metro_manila,
      columns=['City', 'Public_Count'],
      key_on='feature.properties.City',
      fill_color='Greens',
      fill_opacity=1,
      line_opacity=0.2,
      legend_name=f'Number of Public Universities offering {format_legend(course)}'
  ).add_to(m)

  #creates a choropleth map for private universities
  private_univ = folium.Choropleth(
      geo_data=metro_manila,
      name='Private Universities',
      data=metro_manila,
      columns=['City', 'Private_Count'],
      key_on='feature.properties.City',
      fill_color='Reds',
      fill_opacity=1,
      line_opacity=0.2,
      legend_name=f'Number of Private Universities offering {format_legend(course)}'
  ).add_to(m)



  #adds GeoJSON layer with HTML popups
  for _, row in metro_manila.iterrows():
    html = generate_popup_html(row['City'], int(row['Count']), row['University'], row['Link'])
    iframe = folium.IFrame(html, width=300, height='200')
    popup = folium.Popup(iframe, max_width=300)  #sets max width for the popup

    folium.GeoJson(
        data=row.geometry.__geo_interface__,
        style_function=lambda feature: {'fillColor': 'transparent', 'color': 'transparent', 'fillOpacity': 0},
        highlight_function=lambda feature: {'fillColor': '#ffaf00', 'color': 'yellow', 'fillOpacity': 0.5},
        popup=popup
    ).add_to(university)

  for _, row in metro_manila.iterrows():
    #filters public universities for this city
    public_universities = [uni for uni, city, school_type in zip(universities, cities, school_types) if city == row['City'] and 'Public' in school_type]
    public_links = [link for uni, link, city, school_type in zip(universities, links, cities, school_types) if city == row['City'] and 'Public' in school_type]
    html = generate_popup_html(row['City'], int(row['Public_Count']), public_universities, public_links)
    iframe = folium.IFrame(html, width=300, height=200)
    popup = folium.Popup(iframe, max_width=300)
    folium.GeoJson(
        data=row.geometry.__geo_interface__,
        style_function=lambda feature: {'fillColor': 'transparent', 'color': 'transparent', 'fillOpacity': 0},
        highlight_function=lambda feature: {'fillColor': '#ffaf00', 'color': 'yellow', 'fillOpacity': 0.5},
        popup=popup
    ).add_to(public_univ)

  for _, row in metro_manila.iterrows():
    #filters private universities for this city
    private_universities = [uni for uni, city, school_type in zip(universities, cities, school_types) if city == row['City'] and 'Private' in school_type]
    private_links = [link for uni, link, city, school_type in zip(universities, links, cities, school_types) if city == row['City'] and 'Private' in school_type]
    html = generate_popup_html(row['City'], int(row['Private_Count']), private_universities, private_links)
    iframe = folium.IFrame(html, width=300, height=200)
    popup = folium.Popup(iframe, max_width=300)
    folium.GeoJson(
        data=row.geometry.__geo_interface__,
        style_function=lambda feature: {'fillColor': 'transparent', 'color': 'transparent', 'fillOpacity': 0},
        highlight_function=lambda feature: {'fillColor': '#ffaf00', 'color': 'yellow', 'fillOpacity': 0.5},
        popup=popup
    ).add_to(private_univ)

  folium.LayerControl().add_to(m)

  return m

In [ ]:
#creates choopleth map
def map_widget(course, universities, locations, links, school_types):
  map_instance = create_choropleth_map(course, universities, locations, links, school_types)

  with output:
      display(map_instance)

In [ ]:
def content(course, abbreviation, description, career):
  #styled content
  styled_content = f'''
      <div style="
        font-family: Arial, sans-serif;
        font-size: 14px;
        border: 1px solid #ddd;
        padding: 15px;
        border-radius: 10px;
        margin-top: 15px;
        width: 95%;
        text-align: center;
      ">
        <div style="
          font-family: Arial, sans-serif;
          font-size: 18px;
          margin-bottom: 10px;
          color: yellow;
          text-align: center;
      ">
          <strong>{course} ({abbreviation})</strong>
      </div>

      <div style="
        font-family: Arial, sans-serif;
        font-size: 14px;
        text-align: justify;
        margin-top: 10px;
      ">
        <strong>DESCRIPTION:</strong> {description}
      </div>

      <div style="
        font-family: Arial, sans-serif;
        font-size: 14px;
        text-align: justify;
        margin-top: 20px;
      ">
          <strong>CAREER:</strong> {career}
      </div>'''

  display(HTML(styled_content))

In [ ]:
#functions after the user choose a program
def select(change):
  output.clear_output()

  if change['new'] == 'None':
    with output:
        print("Please Select a Degree Program")
  else:
    course = change['new']
    abbreviation = df[df['DEGREE PROGRAM'] == course]['ABBREVIATION'].values[0]
    description = df[df['DEGREE PROGRAM'] == course]['DESCRIPTION'].values[0]
    career = df[df['DEGREE PROGRAM'] == course]['PROGRAM OUTCOMES'].values[0]

    with output:
      content(course, abbreviation, description, career)
      display_button_map = widgets.Button(description="Display Map")
      display_button_map.on_click(lambda b: (map_widget(course, *get_univ(course)), setattr(display_button_map, 'disabled', True)))
      display(display_button_map)

In [ ]:
dropdown = widgets.Dropdown(options=["None"] + name_list, description='Course:')
output = widgets.Output(layout={'margin': '20px 0 0 0'})

dropdown.observe(select, names="value")
display(dropdown)
display(output)

Dropdown(description='Course:', options=('None', 'AB ASIAN STUDIES', 'AB CHRISTIAN EDUCATION', 'AB COMMUNICATI…

Output(layout=Layout(margin='20px 0 0 0'))